In [1]:
import pandas as pd

train = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/train.csv')
test = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/test.csv')
val = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/val.csv')
sample_submission = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/sample_submission.csv')

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
print("Validation Shape:", val.shape)

print("\nTrain Head:\n", train.head())

print("\nMissing values in Train:\n", train.isnull().sum())

print("\nData Types:\n", train.dtypes)

Train Shape: (1914056, 45)
Test Shape: (546874, 44)
Validation Shape: (273437, 45)

Train Head:
    Unique ID  Rider_ID category_x  Circuit_Length_km  Laps  Grid_Position  \
0    1894944      2659      Moto2              4.874    22             17   
1      23438      5205      Moto2              3.875    24              7   
2     939678      7392      Moto3              5.647    25              5   
3    1196312      7894      Moto3              4.810    19              3   
4    1033899      6163     MotoGP              5.809    25             21   

   Avg_Speed_kmh Track_Condition  Humidity_% Tire_Compound_Front  ... air  \
0         264.66             Wet          61                Hard  ...  23   
1         177.56             Wet          77                Soft  ...  12   
2         317.74             Dry          87                Soft  ...  22   
3         321.82             Wet          43                Soft  ...  23   
4         239.92             Wet          47           

In [2]:

import pandas as pd

sample_submission = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/sample_submission.csv')

print(sample_submission.head())
print(sample_submission.columns)

   Unique ID  Lap_Time_Seconds
0     288307            70.667
1     704288           103.497
2     951491            78.400
3    2591721            81.338
4    1202653           108.626
Index(['Unique ID', 'Lap_Time_Seconds'], dtype='object')


In [3]:
import numpy as np 
import pandas as pd


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/burnout-datathon-ieeecsmuj/sample_submission.csv
/kaggle/input/burnout-datathon-ieeecsmuj/val.csv
/kaggle/input/burnout-datathon-ieeecsmuj/train.csv
/kaggle/input/burnout-datathon-ieeecsmuj/test.csv


In [4]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/train.csv')


numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

num_imputer = SimpleImputer(strategy='mean')
df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

cat_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])

print("\nMissing values after filling:")
print(df.isnull().sum())

le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

print("\n✅ Shape:", df.shape)
print("\n✅ Columns:\n", df.columns.tolist())
print("\n✅ Preview:\n", df.head())



Missing values after filling:
Unique ID                          0
Rider_ID                           0
category_x                         0
Circuit_Length_km                  0
Laps                               0
Grid_Position                      0
Avg_Speed_kmh                      0
Track_Condition                    0
Humidity_%                         0
Tire_Compound_Front                0
Tire_Compound_Rear                 0
Penalty                            0
Championship_Points                0
Championship_Position              0
Session                            0
year_x                             0
sequence                           0
rider                              0
team                               0
bike                               0
position                           0
points                             0
shortname                          0
circuit_name                       0
rider_name                         0
team_name                          0
bike_na

In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler

df = pd.read_csv('/kaggle/input/burnout-datathon-ieeecsmuj/train.csv')

num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns
num_imputer = SimpleImputer(strategy='mean')
df[num_cols] = num_imputer.fit_transform(df[num_cols])


cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])


label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  


scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])



cols_to_drop = ['Unique ID', 'Rider_ID']  
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')


print("Shape after preprocessing:", df.shape)


Shape after preprocessing: (1914056, 43)


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

train_df = pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/train.csv")
test_df = pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/test.csv")
val_df = pd.read_csv("/kaggle/input/burnout-datathon-ieeecsmuj/val.csv")

y = train_df['Lap_Time_Seconds']
X = train_df.drop(columns=['Lap_Time_Seconds', 'Unique ID', 'Rider_ID'], errors='ignore')

num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include='object').columns

num_imputer = SimpleImputer(strategy='mean')
X[num_features] = num_imputer.fit_transform(X[num_features])
test_df[num_features] = num_imputer.transform(test_df[num_features])
val_df[num_features] = num_imputer.transform(val_df[num_features])

cat_imputer = SimpleImputer(strategy='most_frequent')
X[cat_features] = cat_imputer.fit_transform(X[cat_features])
test_df[cat_features] = cat_imputer.transform(test_df[cat_features])
val_df[cat_features] = cat_imputer.transform(val_df[cat_features])

label_encoders = {}
for col in cat_features:
    encoder = LabelEncoder()
    X[col] = encoder.fit_transform(X[col])
    test_df[col] = encoder.transform(test_df[col])
    val_df[col] = encoder.transform(val_df[col])
    label_encoders[col] = encoder

scaler = StandardScaler()
X[num_features] = scaler.fit_transform(X[num_features])
test_df[num_features] = scaler.transform(test_df[num_features])
val_df[num_features] = scaler.transform(val_df[num_features])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

model = XGBRegressor(n_estimators=500,
                     learning_rate=0.05,
                     max_depth=10,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     random_state=42,
                     tree_method='hist')
model.fit(X_train, y_train)

y_pred_valid = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
print("Validation RMSE:", rmse)

test_predictions = model.predict(test_df.drop(columns=['Unique ID', 'Rider_ID'], errors='ignore'))

submission = pd.DataFrame({
    'Unique ID': test_df['Unique ID'],
    'Lap_Time_Seconds': test_predictions
})
submission.to_csv("submission.csv", index=False)

Validation RMSE: 5.336489425563644
